In [5]:
# Copyright (C) 2021 Pierre Mourier && Xisco Jiménez Forteza
#
# This program is free software;you can redistribute it and/or modify it
# under the terms of the GNU General Public License as published by the
# Free Software Foundation;either version 3 of the License,or (at your # option) any later version.#
# This program is distributed in the hope that it will be useful,but
# WITHOUT ANY WARRANTY;without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General
# Public License for more details.#
# You should have received a copy of the GNU General Public License along
# with this program;if not,write to the Free Software Foundation,Inc.,# 51 Franklin Street,Fifth Floor,Boston,MA 02110-1301,USA. *)

In [6]:
#This is based on the original code from [2-4] available at [5]; \
#please do refer to these original works
#        and note that the code available at [5] also provides a \
#computation of the angular and radial wavefunctions of the QNMs (not \
#included here).

#     The specificities of this version are described in [6] and \
#include: the use of Leaver's inversions [1] in the calculation of the \
#continued fractions for a more stable recovery of overtones,
#          much larger numbers of steps in the approximations to the \
#continued fractions allowed by the direct implementation of the \
#secant method instead of Mathematica's
#     memory-consuming built-in root-finding algorithm, and a \
# progressive increase of this number of steps over successive \
#iterations along with a convergence criterion, that can be of use
#     for modes for which Leaver's method is less efficient such as \
#near the algebraically special Schwarzschild mode \[Omega] = - 2 \
#\[ImaginaryI].
#The units convention also differs from [1-5] where the mass and \
#maximal dimensionless spin are set to 1/2. This version requires two \
#close but distinct initial guesses for \[Omega]lmn.

#Units are set such that the mass of the black hole is 1.

#Note that this version has not been extensively tested for s \
#\[NotEqual] -2 nor for l \[NotEqual] 2. *)

In [7]:
import numpy as np

In [8]:
# This cell sets the parameters of the mode, of the Kerr black hole (i.e., its spin af), and of the algorithm. 

s=-2;          # Spin weight of the perturbation considered. 
l=2;
m=2;              # l=|s|,|s|+1,|s|+2,... and m=-l, -l+1, ..., l: spheroidal harmonics indices of the QNM.  

af=0.69;   # Dimensionless spin of the Kerr black hole; af in [0,1]. Setting a negative value af in [-1,0[ is also possible and the solution obtained
          # can be equivalently considered as a counter-rotating mode for a black hole of spin |af| up to wlmn --> - wlmn* and Almn --> Almn*,
          # or as a corotating mode of the spheroidal harmonic mode (l,-m) for a black hole of spin |af|. 

nfreq=3;     # Number of inversions (integer) in the computation of Leaver's continued fraction for wlmn.
             # Setting this number to the tone index n=0,1,... of the mode looked for, or to a neighbouring value, should ensure a good stability
             # of the (l,m,n) QNM solution for the root-finding algorithm. It does not ensure however that the solution found is indeed the (l,m,nfreq) mode,
             # as the solution found is rather determined by the closest mode to the initial guesses. 

nang = 0;    # Number of inversions (integer) in the computation of Leaver's continued fraction for Almn.
             # It should be kept at nang=0 for a more stable recovey of (l=2,m,n) modes. 

winit=0.37-0.62j;        # Initial guess for the complex QNM frequency. 
winit2=0.40-0.64j;       # Second initial guess for the frequency (necessary for the secant method).
                         # It should be rather close to, but distinct from, the first initial guess above. 

Almninit=l*(l+1)-s*(s+1);  # Initialisation of Almn to its Schwarzschild value for this choice of s and l.
                           # In most cases, this should not need to be modified. 


# The continued fractions for \[Omega]lmn and Almn are built at each iteration ie=0,1,... with NITMAX(ie) terms, with NITMAX(ie) = nitFractionsStart * nitFractionsFactor^ie. This value of NITMAX may optionally be reduced by a factor nitFractionsAngDecrFactor (e.g. setting this parameter to 10 or 20) for the angular sector for a moderate gain of computation time, as the computation of Almn to a given precision typically requires much less terms in its continued fraction than the computation of \[Omega]lmn.
# Once at least niterMin iterations have been performed,this loop over (\[Omega]lmn,Almn) stops as soon as precisionThreshold is met on the \[Omega]lmn and Almn results over two consecutive iterations. If this convergence criterion fails to be met, the loop is stopped anyway after niterMax iterations. 
nitFractionsStart = 500;
nitFractionsFactor = 1.2;
nitFractionsAngDecrFactor = 1;
niterMax =70;
niterMin = 5;
precisionThreshold = 3*10**(-11);

# The following definitions of additional dependent parameters should not be modified. 

k1=1/2*np.abs(m-s);
k2=1/2*np.abs(m+s);
k1x2p1 = 2*k1+1;
k1pk2 = k1 + k2;
cang2 = s*(s+1)-k1pk2*(k1pk2+1);

br=np.sqrt(1-af**2);

def gamma(n,c2,c4):
    return n**2+(c2-3)*n+c4-c2+2
def beta(n,c1,c3): 
    return -2*n**2+(c1+2)*n+c3
def alpha(n,c0):
    return n**2+(c0+1)*n+c0
def Gamma_ang(n,cang0):
    return cang0*(n+k1pk2+s)
def Beta_ang(n,cang0,cang1):
    return n*(n-1)+2*n*(k1pk2+1-cang0)-cang1
def Alpha_ang(n):
    return -2*(n+1)*(n+k1x2p1)

def ccoeffs(w,Sep):
    auxcoeff=(2*w-af*m)/br
    
    res = [1-s-2j*w-1j*auxcoeff,
           -4+4j*w*(2+br)+2j*auxcoeff,
           s+3-6j*w-1j*auxcoeff, 
           w**2*(16+8*br-af**2)-2*af*m*w-s-1+(4+2*br)*1j*w-Sep+(4*w+1j)*auxcoeff,
           s+1-8*w**2-(4*s+6)*1j*w-(4*w+1j)*auxcoeff]
           
    return res

def cang0f(w):
    return 2*af*w

def cang1f(w,Sep):
    return 2*af*w*(k1x2p1+s)+cang2+af**2*w**2+Sep;

In [9]:
ie=0;
Almn = Almninit
witer=np.zeros(niterMax+1,dtype=complex)
Almniter=np.zeros(niterMax+1,dtype=complex)

while ie<niterMin or (niterMin <=ie < niterMax and max(np.abs(witer[ie-1]-witer[ie-2])+np.abs(witer[ie-2]-witer[ie-3]),np.abs(Almniter[ie-1]-Almniter[ie-2])+np.abs(Almniter[ie-2]-Almniter[ie-3]))>=precisionThreshold):
                                                                                                                  
    NITMAX=round(nitFractionsStart*nitFractionsFactor**ie);

    print(ie+1);
    
    def Leaver31(w,Sep,ninv):
        
        ci=ccoeffs(w,Sep)
        c0,c1,c2,c3,c4 = ci
        
        Rn=-1
        n=NITMAX
        while n>ninv:
            Rn = gamma(n,c2,c4)/(beta(n,c1,c3)-alpha(n,c0)*Rn)
            n=n-1
            
        return Rn
    
    def Leaver33(w,Sep,ninv):
        ci = ccoeffs(w,Sep)
        c0,c1,c2,c3,c4 = ci
        
        Rn=c3
        n=0
        while n<ninv:
            Rn=beta(n+1,c1,c3)-gamma(n+1,c2,c4)*alpha(n,c0)/Rn
            n=n+1
            
        res=Rn/alpha(ninv,c0)-Leaver31(w,Sep,ninv)

        return res
    
    
    i=0
    w1=winit
    w2=winit2
    fw1=Leaver33(winit,Almn,nfreq)
    while i<100 and np.abs(fw1)>=10**(-13):
        fw2=Leaver33(w2,Almn,nfreq)
        w3 = w1-fw1/(fw2-fw1)*(w2-w1);
        w1 = w2
        w2 = w3
        fw1=fw2
        i=i+1
        
    wang=w1
    if np.abs(fw1)>= 10**(-13):
        print("Warning: \[Omega]lmn: root not found!")
    witer[ie]=wang
     
    def Leaver31Ang(w,Sep,ninv):
        cang0 = cang0f(w)
        cang1 = cang1f(w,Sep)
        Rn=-1
        n=round(NITMAX/nitFractionsAngDecrFactor)
       
        while n>ninv:
            Rn=Gamma_ang(n,cang0)/(Beta_ang(n,cang0,cang1)-Alpha_ang(n)*Rn)
            n=n-1
        
        return Rn
            

    def Leaver33Ang(w,Sep,ninv):
        cang0 = cang0f(w)
        cang1 = cang1f(w,Sep)
        n=0; Rn=-cang1;
        
        while n<ninv:
            Rn=Beta_ang(n+1,cang0,cang1)-Gamma_ang(n+1,cang0)*Alpha_ang(n)/Rn
            n=n+1
        
        return Rn/Alpha_ang(ninv)-Leaver31Ang(w,Sep,ninv)
 
    i=0
    Almn1=0.999*Almn
    Almn2=1.001*Almn
    fAlmn1=Leaver33Ang(wang,0.999*Almn,nang)
    
    while i<100 and np.abs(fAlmn1)>=10**(-13):
        fAlmn2=Leaver33Ang(wang,Almn2,nang)
        Almn3 = Almn1-fAlmn1/(fAlmn2-fAlmn1)*(Almn2-Almn1)
        Almn1 =Almn2
        Almn2=Almn3
        fAlmn1=fAlmn2
    
    Almn=Almn1
    
    if np.abs(fAlmn1)>= 10**(-13):
        print("Warning: Almn: root not found!")
        
    Almniter[ie] = Almn;
    print(Almn)
    
    if ie==0:
        print (max(np.abs(witer[ie]-winit),np.abs(witer[ie]-winit2)), np.abs(Almniter[ie]-Almninit),np.abs(witer[ie]-witer[ie-1]),np.abs(Almniter[ie]-Almniter[ie-1]))
    
    ie=ie+1

1
(3.1119130891063747+1.4216175119463503j)
0.14780471376737306 1.6762143990471845 0.8248066724636728 3.4212569947935965
2
(3.2044218024490005+1.2560097994047301j)
3
(3.1939907629273154+1.2792207601544283j)
4
(3.1951266412136463+1.2760958576432415j)
5
(3.195009081738491+1.276515571831664j)
6
(3.1950203067543135+1.2764595712969882j)
7
(3.195019392122316+1.2764669996524787j)
8
(3.1950194384013484+1.276466019965794j)
9
(3.195019442012937+1.2764661484386144j)
10
(3.195019440274372+1.2764661316865629j)
11
(3.195019440666713+1.2764661338584284j)
12
(3.1950194405940326+1.2764661335784961j)
13
(3.1950194406062895+1.2764661336143588j)
14
(3.1950194406043337+1.2764661336097929j)
15
(3.195019440604635+1.2764661336103706j)


In [10]:
[np.abs(af),witer[ie-1].real,witer[ie-1].imag,Almniter[ie-1].real,Almniter[ie-1].imag,np.abs(witer[ie-1]-witer[ie-2])+np.abs(witer[ie-2]-witer[ie-3]),np.abs(Almniter[ie-1]-Almniter[ie-2])+np.abs(Almniter[ie-2]-Almniter[ie-3]),ie]

[0.69,
 0.465789718551652,
 -0.5875924946900086,
 3.195019440604635,
 1.2764661336103706,
 2.547526125530055e-12,
 5.6186509793152694e-12,
 15]